In [1]:
import os

os.environ["SERPAPI_API_KEY"] = "3d009ed0a9764853b3e75ee2c70ceb3aabd92b810456307c5af711559ec4aff7"

In [2]:
from transformers import Tool, load_tool, ReactCodeAgent, HfEngine
# Import tool from LangChain
from langchain.agents import load_tools

# Import tool from Hub
image_generation_tool = load_tool("m-ric/text-to-image")


search_tool = Tool.from_langchain(load_tools(["serpapi"])[0])

llm_engine = HfEngine("meta-llama/Meta-Llama-3-70B-Instruct")
# Initialize the agent with both tools
agent = ReactCodeAgent(tools=[image_generation_tool, search_tool], llm_engine=llm_engine)

# Run it!
#agent.run("Generate me a photo of the car that James bond drove in the latest movie.")

/Users/freddy/sources/components/chatbot-with-tools/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You're loading a tool from the Hub from None. Please make sure this is a source that you trust as the code within that tool will be executed on your machine. Always verify the code of the tools that you load. We recommend specifying a `revision` to ensure you're loading the code that you have checked.


ImportError: Could not import serpapi python package. Please install it with `pip install google-search-results`.

In [ ]:
import json

json.dump(agent.logs, open("logs.json", "w"), indent=2)

In [ ]:
from transformers.agents import Agent
from gradio.data_classes import GradioModel
from typing import Literal, List, Generator, Optional
from threading import Thread
import time

class OpenAIMessage(GradioModel):
    role: Literal["system", "user", "assistant", "tool"]
    content: str
    reasoning: bool = False
    tool_name: Optional[str] = None


In [ ]:
def stream_from_agent(
    agent: Agent,
    prompt: str
) -> Generator[List[OpenAIMessage], None, None]:
    """Run Python code in a process and capture logs in real-time to yield them."""

    thread = Thread(target=agent.run, args=(prompt,))
    num_messages = 0

    # Start process and pull logs while it runs
    thread.start()
    while thread.is_alive():
        if len(agent.logs) > num_messages:
            new_messages = agent.logs[num_messages:]
            for message in new_messages:
                if not len(message):
                    continue
                if message.get("rationale"):
                    yield OpenAIMessage(
                        role="assistant",
                        content=message["rationale"],
                        reasoning=True
                    )
                if message.get("tool_call"):
                    yield OpenAIMessage(
                        role="tool",
                        tool_name=message["tool_call"]["tool_name"],
                        content=message['tool_call']["tool_arguments"],
                        reasoning=True
                    )
                num_messages = len(agent.logs)
        time.sleep(0.1)

    thread.join(0.1)

    if len(agent.logs) > num_messages:
        new_messages = agent.logs[num_messages:]
        for message in new_messages:
            if message.get("rationale"):
                yield OpenAIMessage(
                    role="assistant",
                    content=message["rationale"],
                    reasoning=True
                )
            if message.get("tool_call"):
                yield OpenAIMessage(
                    role="tool",
                    tool_name=message["tool_call"]["tool_name"],
                    content=message.get("tool_arguments", ""),
                )
        num_messages = len(agent.logs)

In [ ]:
messages = []
for msg in stream_from_agent(agent, "Generate me a photo of a cartoon cat."):
    messages.append(msg)
    print("MSG", msg)
print(messages)

In [ ]:
agent.outputs

In [ ]:
agent.logs[2]

In [ ]:
agent.logs[3]

In [ ]:
agent.run("What kind of car is this?")

In [ ]:
agent.logs

In [ ]:
agent.run??